# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Análisis exploratorio y curación de datos

### Trabajo práctico entregable - Grupo 22 - Parte 1

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago
- Spano, Marcelo

**Seguimiento:** Meinardi, Vanesa

---

## Librerías

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine, text

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None  # default='warn'

sns.set_context('talk')
sns.set_theme(style='white')

## Acerca de los datasets

El dataset a utilizar proviene de la [compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia. Particularmente, utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Este [dataset](https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv) está disponible en internet, desde donde lo usaremos.

Por otro lado, vamos a aumentar los datos presentes en dicho conjunto utilizando un dataset similar: las publicaciones de la plataforma AirBnB en Melbourne en el año 2018. El objetivo es estimar con mayor precisión el valor del vecindario de cada propiedad. Este otro [dataset](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv), también disponible en internet, es un conjunto de datos de *scrapings* del sitio realizado por [Tyler Xie](https://www.kaggle.com/tylerx), también disponible en una competencia de Kaggle.

---
# Ejercicio 1 - SQL

## 1) Conexión

Para poder ejecutar consultas simples en SQL con SQLAlchemy, primero debemos crear un ***engine*** : es el punto de partida para cualquier aplicación que hagamos de SQLAlchemy, proporcionando una forma de conectarse e interactuar con la base de datos. El mismo provee además:
- Una ***connection pool***: conjunto de conexiones a la base de datos que permanecen activas por largos períodos de tiempo y se pueden reutilizar eficientemente, previniendo el *overhead* que deviene de la creación de nuevas conexiones, y aumentando la velocidad de funcionamiento.
- Un **dialecto**: SQLAlchemy puede trabajar con muchos tipos de bases de datos, siendo cada uno de estos tipos un dialecto diferente (MySQL, PostgreSQL, Oracle, SQLite, etcétera).

En nuestro caso el dialecto será SQLite y la ingesta de datos se hará en la base de datos database.sqlite3, por lo que instanciamos el *engine* de la siguiente manera:

In [ ]:
# echo flag logs the SQL queries executed by the engine. It’s helpful for
# debugging purposes (True), but don’t use it in a production environmen (False)
engine = create_engine('sqlite:///database.sqlite3', echo=True)

## 2) Ingesta de datos

### Lectura de datos

#### Datos de la competencia Kaggle

Leemos los datos de la competencia Kaggle utilizando pandas. Vemos que en total consta de 13.580 registros con respuestas a 21 variables diferentes.

In [ ]:
# Lectura del csv
url_kag = 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv'
melb_df = pd.read_csv(url_kag)
total_ans_kag = len(melb_df)
print(f'Cantidad de respuestas en el dataset de Kaggle: {total_ans_kag}')
display(melb_df[:3])

In [ ]:
print('--- Información disponible en el dataset de Kaggle ---')
keys_kag = melb_df.keys()
print(f'Contiene un total de {len(keys_kag)} columnas:')
for k in range(len(keys_kag)):
    print(f'{k+1}) {keys_kag[k]}')

#### Datos de Airbnb

Leemos los datos de Airbnb utilizando pandas. Vemos que en total consta de 22.895 registros con respuestas a 84 variables diferentes, *i.e.* tiene 9.315 registros más que en el dataset de Kaggle y responde a 63 variables más.

In [ ]:
# Lectura del csv
url_air = 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv'
airbnb_df = pd.read_csv(url_air)
total_ans_air = len(airbnb_df)
print(f'Cantidad de respuestas en el dataset de Airbnb: {total_ans_air}')
display(airbnb_df[:3])

In [ ]:
print('--- Información disponible en el dataset de Airbnb ---')
keys_air = airbnb_df.keys()
print(f'Contiene un total de {len(keys_air)} columnas:')
for k in range(len(keys_air)):
    print(f'{k+1}) {keys_air[k]}')

Vamos a reducir el dataframe y quedarnos sólo con aquellas columnas que consideramos relevantes para el análisis que pretendemos hacer. Coincide que son 21 variables, pero no necesariamente la relación es 1:1 con las variables de Kaggle.

In [ ]:
int_cols_air = [
    'host_location', 'host_neighborhood', 'street', 'neighborhood', 'city',
    'suburb', 'state', 'zipcode', 'latitude', 'longitude', 'is_location_exact',
    'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
    'beds', 'bed_type', 'price', 'weekly_price', 'monthly_price'
]

airbnb_df = airbnb_df[int_cols_air]
display(airbnb_df[:3])

### Procesamiento de códigos postales

Queremos combinar los datos de Airbnb con los datos de Kaggle. Para ello utilizaremos el código postal como clave común: 'Postcode' en melb_df y 'zipcode' en airbnb_df. Antes que anda, debemos asegurarnos que las columnas se encuentren limpias y con un formato común.

Por un lado, vemos que 'Postcode' en melb_df tiene un formato común: son todos float con un 1 decimal. Vamos a pasarlos todos a enteros.

In [ ]:
print('Formato original de los datos:')
display(melb_df['Postcode'].value_counts().iloc[:10])
print('---------------------------------------------------------------------\n')
melb_df['postcode_int'] = melb_df['Postcode'].fillna(0).astype('int')
print('Datos pasados a enteros:')
display(melb_df['postcode_int'].value_counts().iloc[:10])

Por otra parte , vemos que 'zipcode' en airbnb_df tiene uan mezcla de formatos: algunos son float con un 1 decimal y otros son enteros. Vamos a pasarlos todos a enteros.

In [ ]:
print('Formato original de los datos:')
display(airbnb_df['zipcode'].value_counts()[:10])
print('---------------------------------------------------------------------\n')
# Se estandariza el tipo de datos para la columna zipcode
airbnb_df['zipcode'] = pd.to_numeric(airbnb_df.zipcode, errors='coerce')
airbnb_df['zipcode_int'] = airbnb_df['zipcode'].fillna(0).astype('int')
print('Datos pasados a enteros:')
display(airbnb_df['zipcode_int'].value_counts()[:10])


### Ingesta

Transcribimos todos los registros de melb_df a la tabla "Kaggle" de la base de datos SQL creada previamente.

In [ ]:
melb_df.to_sql('kaggle', con=engine, if_exists="replace")

Transcribimos todos los registros de airbnb_df a la tabla "airbnb" de la base de datos SQL creada previamente.

In [ ]:
airbnb_df.to_sql('airbnb', con=engine, if_exists="replace")

## 3) Consultas

<span style="color:green;font-size:18px">
    Consigna >>>>
</span>

Implementar consultas en SQL que respondan con la siguiente información:

- cantidad de registros totales por ciudad.
- cantidad de registros totales por barrio y ciudad.

<span style="color:green;font-size:18px">
    <<<< Consigna
</span>

In [ ]:
# #cantidad de registros por ciudad:
query_c = "SELECT price FROM survey2 GROUP BY city"
#solo por ciudad y barrio:
query_cb = "SELECT price FROM survey2 GROUP BY city and neighborhood"

con = engine.connect()
sql_text = text(query_cb)
result = con.execute(sql_text)

Se define una función para ejecutar queries con la conexión creada previamente:

In [ ]:
def execute_query(query):
  with engine.connect() as con:
    rs = con.execute(text(query))
    df_rs = pd.DataFrame(rs.fetchall())
  return df_rs

Vemos la cantidad de registros totales por ciudad agrupando por la columna CITY:

In [ ]:
query_city = """SELECT CITY AS CIUDAD, COUNT(*) AS CANT_REGISTROS 
                FROM AIRBNB
                GROUP BY CITY"""
df_city = execute_query(query_city)
df_city.head(10)

Vemos la cantidad de registros totales por barrio y ciudad agrupando por las columnas NEIGHBORHOOD y CITY:

In [ ]:
query_neighborhood_city = """SELECT NEIGHBORHOOD AS BARRIO, CITY AS CIUDAD, COUNT(*) AS CANT_REGISTROS 
                              FROM AIRBNB
                              GROUP BY NEIGHBORHOOD, CITY"""
df_neighborhood_city = execute_query(query_neighborhood_city)
df_neighborhood_city.tail(10)

## 4. Combinación

<span style="color:green;font-size:18px">
    Consigna >>>>
</span>

Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL  para obtener un resultado similar a lo realizado con Pandas en clase.

<span style="color:green;font-size:18px">
    <<<< Consigna
</span>

In [ ]:
# Se realiza una subquery en la que se define AIRBNB_AGG con las agregaciones generadas en clase con Pandas.
# AIRBNB_AGG es joineada con la tabla original PROP_MELB
query_join = """WITH AIRBNB_AGG AS (
                  SELECT ZIPCODE, 
                    AVG(PRICE) AS AIRBNB_PRICE_MEAN,
                    COUNT(PRICE) AS AIRBNB_RECORD_COUNT,
                    AVG(WEEKLY_PRICE) AS AIRBNB_WEEKLY_PRICE_MEAN,
                    AVG(MONTHLY_PRICE) AS AIRBNB_MONTHLY_PRICE_MEAN
                  FROM AIRBNB
                  GROUP BY ZIPCODE
                )
                SELECT * FROM PROP_MELB A
                LEFT JOIN AIRBNB_AGG B 
                ON A.Postcode = B.zipcode"""
df_join = execute_query(query_join)

In [ ]:
df_join.sample(10)

---
# Ejercicio 2 - Pandas

<span style="color:green;font-size:18px">
    Consigna >>>>
</span>

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.

<span style="color:green;font-size:18px">
    <<<< Consigna
</span>

## 1. 

<span style="color:green;font-size:18px">
    Consigna >>>>
</span>

Seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad. Justificar las columnas seleccionadas y las que no lo fueron.
 - Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.

<span style="color:green;font-size:18px">
    <<<< Consigna
</span>

Leemos el dataset nuevamente con Pandas

In [ ]:
df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
df[:3]

Separamos las columnas categóricas de las numéricas

In [ ]:
cols = df.columns
num_cols = ['Rooms', 'Price', 'Distance', 'Bedroom2', 'Bathroom',
       'Car', 'Landsize', 'BuildingArea', 'YearBuilt',
       'Lattitude', 'Longtitude', 'Propertycount']
cat_cols = [x for x in cols if x not in num_cols and x != 'index']

### Análisis de variables numéricas

Vemos la correlación de las variables numéricas con el precio

In [ ]:
df[num_cols].corr()['Price']

Se realiza un scatterplot del Precio en función de todas las variables numéricas

In [ ]:
for col in num_cols:
  plt.figure(figsize=(8,3))
  plt.scatter(df[col], df['Price'])
  plt.axvline(df[col].mode()[0], color='r')
  plt.grid()
  plt.title(col)
  plt.show()

Outliers:
- En la variable Bedroom2 vemos que tiene un valor extremo = 20.
- En la variable Landsize vemos que tiene un valor extremo > 400000
- En la variable BuildingArea vemos que tiene un valor extremo > 40000
- En la variable YearBuilt vemos que tiene un valor extremo en 1200

Quitamos estos valores extremos

In [ ]:
df = df[df['Bedroom2'] < 20]
df = df[df['Landsize'] < 400000]
df = df[df['BuildingArea'] < 40000]
df = df[df['YearBuilt'] > 1200]

Veamos nuevamente la correlación

In [ ]:
corr = df_join[num_cols].corr()['Price']
corr

Realizamos boxplots para las variables discretas

In [ ]:
discrete_cols = ['Rooms', 'Bedroom2', 'Bathroom', 'Car']
for col in discrete_cols:
  df_var = df[[col, 'Price']]
  df_var['count'] = df_var[col].astype(str)
  plt.figure(figsize=(8,3))
  sns.boxplot(data=df_var, x='Price', y='count')
  plt.title(col)
  plt.show()

Separamos a las variables continuas en 6 cuantiles y realizamos los boxplots del Precio.

Se eligen 6 ya que para más cuantiles el código da el siguiente error:

"ValueError: Bin labels must be one fewer than the number of bin edges"

In [ ]:
continuous_cols = [x for x in num_cols if x not in discrete_cols and x!='Price']
for col in continuous_cols:
  df_var = df[[col, 'Price']]
  df_var['quantile'] = pd.qcut(df_var[col], 6, labels=['1','2','3','4','5','6'])
  plt.figure(figsize=(8,3))
  sns.boxplot(data=df_var, x='Price', y='quantile')
  plt.title(col)
  plt.show()

Viendo la correlación y los gráficos anteriores tomamos deciciones respecto a cada columna:


*   Rooms: La correlación con el Precio es de 0.52 por lo que consideramos que es importante para la predicción del valor de la propiedad.
*   Bedroom2: La correlación con el Precio es de 0.5 por lo que consideramos que es importante para la predicción del valor de la propiedad.
*   Bathroom: La correlación con el Precio es de 0.49 por lo que consideramos que es importante para la predicción del valor de la propiedad.
*   Car: La correlación Con el Precio es de 0.25. Viendo el scatterplot y el boxplot de la variable Car no se ve una relación clara con el Precio por lo que la descartamos.
*   Distance: Si bien la correlación con el Precio es relativamente baja (-0.16), se puede observar que para valores en el scatterplot que para valores menores a 20, el precio alcanza los valores más altos mientras que para valores mayores a 20 el precio permanece acotado a valores pequeños. Por esta razón la consideramos en el dataset.
*   Landsize: Se puede observar en el boxplot que las distribuciones son diferentes para cada cuantil, por lo que consideramos que es una variable importante para predecir el valor de la propiedad y la consideramos en el dataset.
*   BuildingArea: Sucede lo mismo que en la variable Landsize, además que posee una correlación alta con el Precio, por lo que la consideramos en el dataset. 
*   YearBuilt: En el boxplot se puede observar que para cuantiles más bajos (1 y 2) el Precio alcanza valores mayores que para el resto de los cuantiles. Además la correlación es negativa, lo que tiene sentido suponiendo que, cuando más vieja es la propiedad, esta pierde valor. La consideramos en el dataset.
*   Lattitude: En el boxplot se puede observar que para cuantiles intermedios (2 y 3) el Precio alcanza valores mayores que para el resto de los cuantiles. La consideramos en el dataset.
*   Longitude: En el boxplot se puede observar que para cuantiles intermedios (4 y 5) el Precio alcanza valores mayores que para el resto de los cuantiles. La consideramos en el dataset.
*   Propertycount: La correlación Con el Precio es de -0.05. Viendo el scatterplot y el boxplot de la variable Propertycount no se ve una relación clara con el Precio por lo que la descartamos.



In [ ]:
selected_num_cols = ['Rooms', 'Bedroom2', 'Bathroom', 'Distance', 'Landsize',
                    'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude']

### Análisis de variables categóricas

In [ ]:
cat_cols

Veamos la cardinalidad de cada una y el porcentaje de repeticiones que tienen las 10 categorías con mayor cantidad:

In [ ]:
for col in cat_cols:
  print(col)
  print('Cardinalidad:', df[col].nunique())
  print(100*df[col].value_counts(normalize=True).iloc[:10])
  print()

* Suburb: Tiene 300 valores diferentes y la categoría que posee la mayor cantidad de casos solo tiene el 2,5% por lo que la descartamos.
* Address: La descartamos ya que la dirección es única para cada propiedad y no es representativa.
* Type: Tiene 3 valores únicos, la consideramos en el dataset.
* Method: Posee 5 valores únicos, la consideramos en el dataset.
* SellerG: Posee 214 valores únicos, pero vemos que algunos de estos valores poseen un gran porcentaje, como Nelson que tiene el 12% o Jellis con el 10%. La consideramos en el dataset.
* Date: No consieramos que sea relevante, la descartamos.
* Postcode: Tiene 190 valores diferentes y la categoría que posee la mayor cantidad de casos solo tiene el 2,5% por lo que la descartamos.
* CouncilArea: Posee 31 valores únicos, pero vemos que algunos de estos valores poseen un gran porcentaje de casos, como Moreland que tiene el 10% o Boroondara con el 9%. La consideramos en el dataset.
* Regionname: Tiene 8 valores únicos, la consideramos en el dataset.

In [ ]:
preselected_cat_cols = ['Type', 'Method', 'SellerG', 'CouncilArea', 'Regionname']

En el caso de las variables de mayor cardinalidad, CouncilArea y SellerG, creamos una categoría nueva "Other" para agrupar a los valores que posean menos del 1% de los casos.

In [ ]:
for col in ['SellerG', 'CouncilArea']:
  value_counts = 100*df[col].value_counts(normalize=True)
  lower_values = value_counts[value_counts<1].index.tolist()
  df[col] = df[col].apply(lambda x: 'Other' if x in lower_values else x)

Veamos la nueva cardinalidad de ambas variables

In [ ]:
df['SellerG'].nunique()

In [ ]:
df['CouncilArea'].nunique()

Ahora veamos los boxplots para las variables categóricas preseleccionadas

In [ ]:
for col in preselected_cat_cols:
  values = df[col].value_counts(normalize=True).index[:10].tolist()
  plt.figure(figsize=(8,4))
  sns.boxplot(data=df[df[col].isin(values)], y=col, x='Price')
  plt.show()

En todos los casos vemos que hay valores para los cuales el precio tiende a tener precios mayores que en otros por lo que seleccionamos todas estas variables

In [ ]:
selected_cat_cols = preselected_cat_cols

### Unificamos variables seleccionadas numéricas y categóricas

In [ ]:
selected_cols = selected_num_cols + selected_cat_cols

In [ ]:
selected_cols

In [ ]:
df = df[selected_cols + ['Price', 'Postcode']]

In [ ]:
df.head()

## 2. 

<span style="color:green;font-size:18px">
    Consigna >>>>
</span>

Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico. 
  1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.
  1. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
  2. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

<span style="color:green;font-size:18px">
    <<<< Consigna
</span>

In [ ]:
airbnb_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv')

In [ ]:
# Se estandariza el tipo de datos para la columna zipcode
airbnb_df['zipcode'] = pd.to_numeric(airbnb_df.zipcode, errors='coerce')

In [ ]:
airbnb_df.head(3)

In [ ]:
sns.scatterplot(airbnb_df.zipcode.value_counts().values)
plt.axhline(100, color='g')
plt.axhline(50, color='r')

Incluimos los zipcodes que tengan una cantidad mayor o igual a 100 registros para que la información agregada sea relevante

In [ ]:
value_counts = airbnb_df.zipcode.value_counts()
value_counts = value_counts[value_counts>=100]
airbnb_df = airbnb_df[airbnb_df['zipcode'].isin(value_counts.index.tolist())]

In [ ]:
airbnb_price_by_zipcode = airbnb_df.groupby('zipcode')\
  .agg({'price': ['mean', 'count'], 'weekly_price': 'mean',
        'monthly_price': 'mean', 'number_of_reviews': ['sum', 'mean'],
        'review_scores_rating': ['min', 'max', 'mean']})\
  .reset_index()
# Flatten the two level columns
airbnb_price_by_zipcode.columns = [
  ' '.join(col).strip()
  for col in airbnb_price_by_zipcode.columns.values]

In [ ]:
airbnb_price_by_zipcode.columns

In [ ]:
# Rename columns
airbnb_price_by_zipcode = airbnb_price_by_zipcode.rename(
    columns={'price mean': 'airbnb_price_mean',
             'price count': 'airbnb_record_count',
             'weekly_price mean': 'airbnb_weekly_price_mean',
             'monthly_price mean': 'airbnb_monthly_price_mean',
             'number_of_reviews sum': 'airbnb_number_of_reviews_sum', 
             'number_of_reviews mean': 'airbnb_number_of_reviews_mean', 
             'review_scores_rating min': 'airbnb_review_scores_rating_min', 
             'review_scores_rating max': 'airbnb_review_scores_rating_max',
             'review_scores_rating mean': 'airbnb_review_scores_rating_mean'}
)

In [ ]:
airbnb_price_by_zipcode.head()

In [ ]:
df_join = df.merge(
    airbnb_price_by_zipcode, how='left',
    left_on='Postcode', right_on='zipcode'
)
df_join.sample(5)

## 3. 

Se podría utilizar la variable Suburb que se encuentra en ambos datasets y posee una cardinalidad mayor al código postal en ambos casos

In [ ]:
print('Cardinalidad Código postal:', melb_df.Postcode.nunique())
print('Cardinalidad Suburbio:', melb_df.Suburb.nunique())

In [ ]:
print('Cardinalidad Código postal:', airbnb_df.zipcode.nunique())
print('Cardinalidad Suburbio:', airbnb_df.suburb.nunique())

Otra opción sería utilizar la latitud y longitud que se encuentra en ambos datasets y encontrar todas las propiedades que se encuentren a cierta distancia.

---
# Ejercicio 3 - Guardado final

<span style="color:green;font-size:18px">
    Consigna >>>>
</span>

Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.

<span style="color:green;font-size:18px">
    <<<< Consigna
</span>

In [ ]:
df_join.head()

Guardamos el dataset en un archivo csv:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/new_dataset.csv'

df_join.to_csv(path)

---
# Ejercicios Opcionales

<span style="color:green;font-size:18px">
    Consigna >>>>
</span>

1. Armar un script en python (archivo .py) [ETL](https://towardsdatascience.com/what-to-log-from-python-etl-pipelines-9e0cfe29950e) que corra los pasos de extraccion, transformacion y carga, armando una funcion para cada etapa del proceso y luego un main que corra todos los pasos requeridos.

2. Armar un DAG en Apache Airflow que corra el ETL. (https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html)

<span style="color:green;font-size:18px">
    <<<< Consigna
</span>